In [1]:
import cv2
import os
import time
from matplotlib import pyplot as plt
import numpy as np
import onnxruntime
classes = ["b4_bracket", "blue_cable_tie","bolt_v1","bolt_v3","bolt_v4","bolt_v5","green_disk","screw_v1","screw_v2","transparent_cap","welding_seam","welding_spot","yellow_cable_tie","yellow_cap"]
model_path = "/home/tandonsa/PycharmProjects/FCM-model_dev/yolov5/runs/train/exp/weights/best.onnx"

In [2]:
session = onnxruntime.InferenceSession(model_path)
IN_IMAGE_H = session.get_inputs()[0].shape[2]
IN_IMAGE_W = session.get_inputs()[0].shape[3]
batch_size = session.get_inputs()[0].shape[0]
print(batch_size)
print(IN_IMAGE_H)
print(IN_IMAGE_W)

1
320
320


In [3]:
def detect(img_src,img_shape,model):
    img_in = cv2.resize(img_src, img_shape, interpolation=cv2.INTER_LINEAR)
    img_in = np.transpose(img_in, (2, 0, 1)).astype(np.float32)
    img_in = np.expand_dims(img_in, axis=0)
    img_in /= 255.0
    # Compute
    input_name = model.get_inputs()[0].name
    outputs = model.run(None, {input_name: img_in})
    
    detection_classes,detection_boxes,detection_scores, num_detections = outputs
    print("input image",img_in.shape)
    print("detection_boxes",detection_boxes.shape)
    print("detection_classes",detection_classes.shape)
    print("detection_scores",detection_scores.shape)
    print("num_detections",num_detections.shape)
#     boxes = post_processing(img_in,outputs ,0.4, 0.6)
#     print(boxes)
    return img_in,detection_classes,detection_boxes,detection_scores, num_detections
#     return boxes


def nms_cpu(boxes, confs, nms_thresh=0.5, min_mode=False):
    # print(boxes.shape)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1) * (y2 - y1)
    order = confs.argsort()[::-1]

    keep = []
    while order.size > 0:
        idx_self = order[0]
        idx_other = order[1:]

        keep.append(idx_self)

        xx1 = np.maximum(x1[idx_self], x1[idx_other])
        yy1 = np.maximum(y1[idx_self], y1[idx_other])
        xx2 = np.minimum(x2[idx_self], x2[idx_other])
        yy2 = np.minimum(y2[idx_self], y2[idx_other])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h

        if min_mode:
            over = inter / np.minimum(areas[order[0]], areas[order[1:]])
        else:
            over = inter / (areas[order[0]] + areas[order[1:]] - inter)

        inds = np.where(over <= nms_thresh)[0]
        order = order[inds + 1]
    
    return np.array(keep)
            

def post_processing(img,output,conf_thresh, nms_thresh):
    box_array = output[1]  # (1, 3, 40, 40, 19) # [batch, num, 1, 4]
    print("BOX array shape",box_array.shape)
    confs = output[0] # (1, 6300, 19)   # [batch, num, num_classes]
    print("Confidence shape ",confs.shape) 
    t1 = time.time()
    
    if type(box_array).__name__ != 'ndarray':
        box_array = box_array.cpu().detach().numpy()
        confs = confs.cpu().detach().numpy()
    
    num_classes = confs.shape[2]
    print("total num classes",num_classes)
    
    max_conf = np.max(confs, axis=2)
    max_id = np.argmax(confs, axis=2)
    t2 = time.time()
    
    bboxes_batch = []
    for i in range(box_array.shape[0]):
        argwhere = max_conf[i] > conf_thresh
        l_box_array = box_array[i, argwhere, :]
        l_max_conf = max_conf[i, argwhere]
        l_max_id = max_id[i, argwhere]
        bboxes = []
        # nms for each class
        for j in range(num_classes):
            cls_argwhere = l_max_id == j
            ll_box_array = l_box_array[cls_argwhere, :]
            ll_max_conf = l_max_conf[cls_argwhere]
            ll_max_id = l_max_id[cls_argwhere]

            keep = nms_cpu(ll_box_array, ll_max_conf, nms_thresh)
            
            if (keep.size > 0):
                ll_box_array = ll_box_array[keep, :]
                ll_max_conf = ll_max_conf[keep]
                ll_max_id = ll_max_id[keep]

                for k in range(ll_box_array.shape[0]):
                    bboxes.append([ll_box_array[k, 0], ll_box_array[k, 1], ll_box_array[k, 2], ll_box_array[k, 3], ll_max_conf[k], ll_max_conf[k], ll_max_id[k]])
        
        bboxes_batch.append(bboxes)
        
    t3 = time.time()

    print('-----------------------------------')
    print('       max and argmax : %f' % (t2 - t1))
    print('                  nms : %f' % (t3 - t2))
    print('Post processing total : %f' % (t3 - t1))
    print('-----------------------------------')
    
    return bboxes_batch

def class_colors(names):

    return {name: (
        np.random.randint(0, 255),
        np.random.randint(0, 255),
        np.random.randint(0, 255)) for name in names}

def plot_boxes_cv2(img, boxes, savename=None, class_names=None, color=None,save_cords=None):
    img = np.copy(img)
    color = class_colors(class_names)
    width = img.shape[1]
    height = img.shape[0]
    
    for i in range(len(boxes[0])):
        box = boxes[0]
        x1 = int(box[i][0] * width)
        y1 = int(box[i][1] * height)
        x2 = int(box[i][2] * width)
        y2 = int(box[i][3] * height)
        if class_names:
            cls_conf = box[i][5]
            cls_id = box[i][6]
            cls_label = class_names[cls_id]
            img = cv2.putText(img, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,2, color[cls_label], 2)
        img = cv2.rectangle(img, (x1, y1), (x2, y2),color[cls_label], 5)
        if save_cords:
            fd = open(save_cords,'a+')
            fd.write("{} {} {} {} {}\n".format(class_names[cls_id],x1,y1,x2,y2))
        
    if savename:
        print("save plot results to %s" % savename)
        cv2.imwrite(savename, img)

    return img

In [4]:
testimg_file = '/home/tandonsa/PycharmProjects/FCM-model_dev/yolov5_azureml/src/resources/val/images/NLRRM05J_223283605J200312113415_20200312-121039_front_camera_0_system1.jpg'
test_img = cv2.imread(testimg_file)
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

test_img,det_classes,det_boxes,det_scores, num_det =detect(test_img,img_shape=(IN_IMAGE_H,IN_IMAGE_W),model=session)

# classes = ["b4_bracket", "blue_cable_tie","bolt_v1","bolt_v3","bolt_v4","bolt_v5","green_disk","screw_v1","screw_v2","transparent_cap","welding_seam","welding_spot","yellow_cable_tie","yellow_cap"]
# img = plot_boxes_cv2(test_img,boxes,class_names=classes,savename='predicted.jpg',save_cords='boxes.txt')

# plt.figure(figsize=(15,15))
# plt.imshow(img)

input image (1, 3, 320, 320)
detection_boxes (1, 3, 40, 40, 19)
detection_classes (1, 6300, 19)
detection_scores (1, 3, 20, 20, 19)
num_detections (1, 3, 10, 10, 19)


In [ ]:
batch_size = num_det.shape[0]
for batch in range(0, batch_size):
    c = det_classes[batch][1]
    d = det_boxes[batch][1]
    print(c.shape,d.shape)
    

In [ ]:
# draw results
def draw_detection(draw, d, c):
    width, height = draw.im.size
    # the box is relative to the image size so we multiply with height and width to get pixels.
    top = d[0] * height
    left = d[1] * width
    bottom = d[2] * height
    right = d[3] * width
    top = max(0, np.floor(top + 0.5).astype('int32'))
    left = max(0, np.floor(left + 0.5).astype('int32'))
    bottom = min(height, np.floor(bottom + 0.5).astype('int32'))
    right = min(width, np.floor(right + 0.5).astype('int32'))
    label = coco_classes[c.astype('int32') - 1] # shift to zero element
    label_size = draw.textsize(label)
    text_origin = tuple(np.array([left + 1, top + 1]))
    color = ImageColor.getrgb("yellow")
    thickness = 0
    draw.rectangle([left + thickness, top + thickness, right - thickness, bottom - thickness], outline=color)
    draw.text(text_origin, label, fill=color, font=font)

